### Merge Data Summary
- Performed a left join to merge NOAA weather data to the cbc observations.
- There are still 35757 observations without NOAA weather information. About 39.92% of the cbc_cleaned dataset.
- 3 output files:


1. Complete cbc dataset paired with NOAA wether observations: 'cbc_cleaned_usa_merged.csv'
2. cbc datset subset with observations successfully paired to a NOAA weather observation: 'cbc_cleaned_usa_paired.csv'
3. cbc datset subset with observations not paired to a NOAA weather observation: 'cbc_cleaned_usa_notpaired.csv'

In [1]:
import pandas as pd

In [2]:
#Import cbc cleanse data into pandas dataframe
cbc_cleaned = pd.read_csv('cbc_cleaned_usa_statid.csv')
#Drop the index included in the csv
cbc_cleaned = cbc_cleaned.iloc[:,1:]
cbc_cleaned['count_date']=pd.to_datetime(cbc_cleaned['count_date'])
#Import flattened NOAA data into pandas dataframe
flatten_data1967 = pd.read_csv('flatten_data_from_1900_to_1967.csv')
flatten_data2000 = pd.read_csv('flatten_data_from_1968_to_2000.csv')
flatten_data2019 = pd.read_csv('flatten_data_from_2000_to_2019.csv')
#Get all NOAA data on a single df
flatten_data = pd.concat([flatten_data1967,flatten_data2000,flatten_data2019],ignore_index=True)
#Drop the index included in the csv
flatten_data = flatten_data.iloc[:,1:]
flatten_data['date']=pd.to_datetime(flatten_data['date'])

/Users/feraguilari/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/feraguilari/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [3]:
#Visualize first 5 rows.
flatten_data.head()

,date,id,name,precipitation_value,state,temp_max_value,temp_min_value
0,1900-12-25,CA006139520,WINDSOR RIVERSIDE,0.0,ON,11.0,-50.0
1,1900-12-26,CA006139520,WINDSOR RIVERSIDE,0.0,ON,-22.0,-56.0
2,1900-12-25,RSM00032618,NIKOLSKOYE/BERINGA OSTROV,0.0,,-50.0,-69.0
3,1900-12-26,RSM00032618,NIKOLSKOYE/BERINGA OSTROV,58.0,,10.0,-55.0
4,1900-12-25,USC00010836,BIRMINGHAM,0.0,AL,133.0,22.0


In [4]:
#Visualize first 5 rows.
cbc_cleaned.head()

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,...,max_wind_metric,min_wind_imperial,max_wind_imperial,station_key,circle_coordinates,closest_station_id,closest_station_lat,closest_station_lng,distance,circle_num
0,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(36.61670, -121.9167)","(36.61670, -121.9167)",USC00047916,36.9878,-121.9994,41928.062371,2
1,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(38.175251, -104.519575)","(38.175251, -104.519575)",USW00023068,38.2725,-104.6089,13338.679404,3
2,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(41.67180, -72.94950)","(41.67180, -72.94950)",USC00061536,41.8000,-72.9333,14322.471705,4
3,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(41.11670, -73.40000)","(41.11670, -73.40000)",USC00065892,41.1333,-73.4500,4578.046523,5
4,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(41.88330, -88.06670)","(41.88330, -88.06670)",USC00119221,41.8128,-88.0728,7857.728111,6


In [5]:
# Find duplicate values
flatten_id=pd.DataFrame.duplicated(flatten_data,subset=['id','date'], keep='first')
print('Duplicates: ',flatten_id.sum())

Duplicates:  42972


In [6]:
#Remove duplicate data
flatten_data=flatten_data[~flatten_id]
#Left join flatten data to cbc cleaned data by count date and station id.
merged_data = cbc_cleaned.merge(flatten_data, how='left',left_on=['count_date','closest_station_id'],
                               right_on=['date','id'])
#Visualize first 5 rows.
merged_data.head()

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,...,closest_station_lng,distance,circle_num,date,id,name,precipitation_value,state,temp_max_value,temp_min_value
0,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-121.9994,41928.062371,2,1900-12-25,USC00047916,SANTA CRUZ,0.0,CA,172.0,33.0
1,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-104.6089,13338.679404,3,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,...,-72.9333,14322.471705,4,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-73.4500,4578.046523,5,1900-12-25,USC00065892,NORWALK,0.0,CT,67.0,-44.0
4,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-88.0728,7857.728111,6,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Check for rows that did not pair with NOAA data
missing_vals=merged_data.id.isna().sum()
print('Rows with no pairing: ',missing_vals)
print('As a percentage: ', round(missing_vals/len(merged_data)*100,2),'%')

Rows with no pairing:  35757
As a percentage:  39.92 %


In [8]:
#Get a df with cbc data that was not paired to NOAA data
notpaired_data=merged_data[merged_data.id.isna()].iloc[:,:54]
notpaired_data

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,...,max_wind_metric,min_wind_imperial,max_wind_imperial,station_key,circle_coordinates,closest_station_id,closest_station_lat,closest_station_lng,distance,circle_num
1,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(38.175251, -104.519575)","(38.175251, -104.519575)",USW00023068,38.2725,-104.6089,13338.679404,3
2,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(41.67180, -72.94950)","(41.67180, -72.94950)",USC00061536,41.8000,-72.9333,14322.471705,4
4,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(41.88330, -88.06670)","(41.88330, -88.06670)",USC00119221,41.8128,-88.0728,7857.728111,6
9,La Grange,US-MO,40.050000,-91.500000,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(40.05000, -91.50000)","(40.05000, -91.50000)",USC00231275,40.1433,-91.5158,10464.129047,11
11,Englewood,US-NJ,40.883300,-73.966700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,"(40.88330, -73.96670)","(40.88330, -73.96670)",USC00300961,40.8369,-73.8494,11135.596654,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89550,Spokane,US-WA,47.715062,-117.411182,2016,2016-01-03,63.0,14.0,13.0,23.0,...,6.437078,1.0,4.0,"(47.715062, -117.411182)","(47.715062, -117.411182)",US1WASP0034,47.7152,-117.4547,3256.705327,101630
89551,Erie,US-PA,42.083311,-80.033301,2016,2015-12-19,17.0,1.0,2.0,7.0,...,54.715159,16.0,34.0,"(42.083311, -80.033301)","(42.083311, -80.033301)",US1PAER0002,42.0609,-80.1526,10160.268865,101263
89553,Mineral,US-CA,40.347776,-121.596116,2016,2015-12-17,7.0,1.0,NaN,5.0,...,1.609269,0.0,1.0,"(40.347776, -121.596116)","(40.347776, -121.596116)",USC00045679,40.3458,-121.6092,1130.697364,100059
89555,Aspen,US-CO,39.250006,-106.900001,2013,2012-12-30,16.0,1.0,3.0,5.0,...,12.874155,0.0,8.0,"(39.250006, -106.900001)","(39.250006, -106.900001)",US1COPT0010,39.2679,-106.8618,3845.119580,92777


In [9]:
paired_data=merged_data[merged_data.id.notna()]
paired_data

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,...,closest_station_lng,distance,circle_num,date,id,name,precipitation_value,state,temp_max_value,temp_min_value
0,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-121.9994,41928.062371,2,1900-12-25,USC00047916,SANTA CRUZ,0.0,CA,172.0,33.0
3,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-73.4500,4578.046523,5,1900-12-25,USC00065892,NORWALK,0.0,CT,67.0,-44.0
5,Baldwin,US-LA,29.833300,-91.550000,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-91.5442,1245.124892,7,1900-12-25,USC00163313,FRANKLIN 3 NW,0.0,LA,161.0,6.0
6,Belmont,US-MA,42.383300,-71.166700,1901,1900-12-25,1.0,NaN,NaN,NaN,...,-71.1167,4107.877116,8,1900-12-25,USC00191097,CAMBRIDGE,0.0,MA,83.0,-11.0
7,Greater Boston,US-MA,42.397309,-71.095501,1901,1900-12-26,1.0,NaN,NaN,NaN,...,-71.1167,2336.790124,9,1900-12-26,USC00191097,CAMBRIDGE,0.0,MA,50.0,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89562,Cable,US-WI,46.201471,-91.152765,2016,2015-12-19,5.0,5.0,4.0,4.0,...,-90.9808,13714.194763,101688,2015-12-19,USC00471604,CLAM LAKE 4W,0.0,WI,-100.0,-161.0
89563,Hayward,US-WI,46.010021,-91.290002,2016,2015-12-19,5.0,7.0,4.0,4.0,...,-91.4442,12043.634721,101706,2015-12-19,USW00094973,HAYWARD MUNI AP,0.0,WI,-60.0,-143.0
89564,Copper Center,US-AK,62.019112,-145.369224,2013,2012-12-14,7.0,4.0,7.0,8.0,...,-145.5331,13114.478871,92546,2012-12-14,USC00503304,GLENNALLEN KCAM,0.0,AK,-133.0,-211.0
89565,Washington Island,US-WI,45.383338,-86.883312,2017,2016-12-17,4.0,20.0,2.0,2.0,...,-86.8911,2850.597664,104285,2016-12-17,USC00478905,WASHINGTON IS,140.0,WI,-50.0,-111.0


In [10]:
merged_data.to_csv('cbc_cleaned_usa_merged.csv')
paired_data.to_csv('cbc_cleaned_usa_paired.csv')
notpaired_data.to_csv('cbc_cleaned_usa_notpaired.csv')